*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2022-08-15 19:07:00--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  5.79MB/s    in 5.6s    

2022-08-15 19:07:06 (4.47 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [199]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [200]:
filter_1 = df_ratings['user'].value_counts()
filter_2 = df_ratings['isbn'].value_counts()
df_ratings = df_ratings[~df_ratings['user'].isin(filter_1[filter_1 < 200].index) & ~df_ratings['isbn'].isin(filter_2[filter_2 < 100].index)]

In [201]:
df = pd.merge(df_ratings,df_books,on='isbn')

In [202]:
df.head()

,user,isbn,rating,title,author
0,277427,002542730X,10.000,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
1,3363,002542730X,0.000,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
2,11676,002542730X,6.000,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
3,12538,002542730X,10.000,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
4,13552,002542730X,0.000,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner


In [203]:
combine_book_rating = df.dropna(axis = 0, subset = ['title'])
book_ratingCount = (combine_book_rating.
     groupby(by = ['title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
     [['title', 'totalRatingCount']]
    )
book_ratingCount.head()

,title,totalRatingCount
0,1984,52
1,1st to Die: A Novel,162
2,2nd Chance,123
3,4 Blondes,72
4,A Beautiful Mind: The Life of Mathematical Gen...,46


In [204]:
rating_with_totalRatingCount = combine_book_rating.merge(book_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
rating_with_totalRatingCount.head()

,user,isbn,rating,title,author,totalRatingCount
0,277427,002542730X,10.000,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,80
1,3363,002542730X,0.000,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,80
2,11676,002542730X,6.000,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,80
3,12538,002542730X,10.000,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,80
4,13552,002542730X,0.000,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,80


In [205]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(book_ratingCount['totalRatingCount'].describe())

count   673.000
mean     73.577
std      39.844
min      16.000
25%      48.000
50%      63.000
75%      86.000
max     365.000
Name: totalRatingCount, dtype: float64


In [183]:
popularity_threshold = 200
rating_popular_book= rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_book.head()

,user,isbn,rating,title,author,totalRatingCount
62,276727,0446520802,0.000,The Notebook,Nicholas Sparks,650
63,278418,0446520802,0.000,The Notebook,Nicholas Sparks,650
64,638,0446520802,0.000,The Notebook,Nicholas Sparks,650
65,3363,0446520802,0.000,The Notebook,Nicholas Sparks,650
66,7158,0446520802,10.000,The Notebook,Nicholas Sparks,650


In [206]:
rating_popular_book.head()

,user,isbn,rating,title,author,totalRatingCount
62,276727,0446520802,0.000,The Notebook,Nicholas Sparks,650
63,278418,0446520802,0.000,The Notebook,Nicholas Sparks,650
64,638,0446520802,0.000,The Notebook,Nicholas Sparks,650
65,3363,0446520802,0.000,The Notebook,Nicholas Sparks,650
66,7158,0446520802,10.000,The Notebook,Nicholas Sparks,650


In [185]:
thr = 100
rating_popular_book=rating_popular_book[rating_popular_book.title.duplicated(keep=False).groupby(rating_popular_book.title).transform('sum').gt(thr)]

In [197]:
rating_popular_book.head()

,user,isbn,rating,title,author,totalRatingCount
62,276727,0446520802,0.000,The Notebook,Nicholas Sparks,650
63,278418,0446520802,0.000,The Notebook,Nicholas Sparks,650
64,638,0446520802,0.000,The Notebook,Nicholas Sparks,650
65,3363,0446520802,0.000,The Notebook,Nicholas Sparks,650
66,7158,0446520802,10.000,The Notebook,Nicholas Sparks,650


In [207]:
book_features_df=rating_popular_book.pivot_table(index='title',columns='user',values='rating').fillna(0)
book_features_df.head()

user,8,9,14,16,26,32,42,44,51,67,...,278800,278813,278824,278828,278832,278836,278843,278844,278846,278854
title,,,,,,,,,,,,,,,,,,,,,
1984,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1st to Die: A Novel,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2nd Chance,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
A Bend in the Road,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
A Case of Need,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [208]:
book_features_df_matrix = csr_matrix(book_features_df.values)

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(book_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [144]:

query_index=7

In [153]:
book_features_df.shape[0]

297

In [108]:
title=list(book_features_df.index)


1

In [89]:
book_features_df.loc[book_features_df[0]=='1984']


KeyError: ignored

In [160]:
print(book_features_df_matrix)

  (0, 28)	9.0
  (0, 152)	10.0
  (0, 198)	8.0
  (0, 201)	10.0
  (0, 259)	10.0
  (0, 675)	9.0
  (0, 693)	8.0
  (0, 807)	5.0
  (0, 912)	6.0
  (0, 1050)	3.3333333
  (0, 1372)	9.0
  (0, 1532)	8.0
  (0, 2241)	7.0
  (0, 2893)	10.0
  (0, 3254)	8.0
  (0, 3266)	10.0
  (0, 3731)	7.0
  (0, 3748)	9.0
  (0, 3861)	9.0
  (0, 3916)	10.0
  (0, 4497)	10.0
  (0, 4736)	10.0
  (0, 4785)	10.0
  (0, 4908)	9.0
  (0, 5002)	9.0
  :	:
  (296, 22404)	3.0
  (296, 22408)	8.0
  (296, 22410)	7.0
  (296, 22558)	9.0
  (296, 22585)	7.0
  (296, 22758)	8.0
  (296, 23534)	10.0
  (296, 24032)	8.0
  (296, 24066)	5.0
  (296, 24104)	5.0
  (296, 24274)	6.0
  (296, 24665)	10.0
  (296, 25200)	6.0
  (296, 25430)	9.0
  (296, 25442)	8.0
  (296, 25896)	9.0
  (296, 25929)	5.0
  (296, 26346)	7.0
  (296, 26584)	7.0
  (296, 26653)	8.0
  (296, 26925)	8.0
  (296, 27360)	8.0
  (296, 27363)	9.0
  (296, 27526)	7.0
  (296, 27756)	9.0


In [158]:
book_features_df.values

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [209]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  recommended_books = []
  model_knn = NearestNeighbors(n_neighbors=6, metric="cosine")
  model_knn.fit(book_features_df.values)
  distances, indices = model_knn.kneighbors([book_features_df.loc[book].values], n_neighbors=6)
  for i in range(1,6):
    recommended_books.append([book_features_df.index[indices[0][-i]], distances[0][-i]])
  
  return [book, recommended_books]

In [218]:
def get_recommends(book = ""):
  recommended_books = []
  nbrs = NearestNeighbors(n_neighbors=6, metric="cosine").fit(book_features_df.values)
  distances, indices = nbrs.kneighbors([book_features_df.loc[book].values], n_neighbors=6)
  for i in range(1,6):
    recommended_books.append([book_features_df.index[indices[0][-i]], distances[0][-i]])
  
  return [book, recommended_books]

In [193]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  recommended_books=[]
  distances, indices = model_knn.kneighbors(book_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)
  query_index=title.index(book)
  for i in range(0, len(distances.flatten())):
        x=(book_features_df.index[indices.flatten()[i]], distances.flatten()[i])
        print(list(x))

  return recommended_books

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [219]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['Icy Sparks', 0.92420506], ['Jewel', 0.9230156], ["The Pilot's Wife : A Novel", 0.9172571], ['I Know This Much Is True', 0.9102433], ['The Lovely Bones: A Novel', 0.90959823]]]
You haven't passed yet. Keep trying!


In [214]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

KeyError: ignored